# PPI Workshop : An example of Neural Network Training for Protein-Protein Interaction Prediction with PIPR


## <cite>[PIPR][1]</cite> Presentation : Multifaceted protein–protein interaction prediction based on Siamese residual RCNN 

Chen, Muhao, et al. "Multifaceted protein–protein interaction prediction based on Siamese residual RCNN." Bioinformatics 35.14 (2019): i305-i314.

- For our hands-on training, we will analyse an example of deep neural network, PIPR, that relies only on protein sequences as input to predict its interaction.
- The original code from Chen, Muhao, et al. is available on github <cite>[here][2]</cite>. In this training, we have made some modifications, so that it can easily be understood and tested.

[1]: http://dx.doi.org/10.1093/bioinformatics/btz328
[2]: https://github.com/muhaochen/seq_ppi

## Prerequisites
- Due to the variety of the users work environment (operating system, computational power), a certain number of choices have been made for the purpose of this training.
- The global prerequisites are the following:
```
    python 3.6 or higher
    Tensorflow 2.0 (with GPU support)
    CuDNN
    Keras 2.2.4
```
- Depending on the operating system, installing the GPU environment (tensorflow,Cuda,Keras) can vary. We propose the following links for step by step help on installing these elements, for <cite>[Linux][1]</cite> or for <cite>[Windows][2]</cite>. Note that using the tensorflow/keras environment does not necessarily imply having a GPU, you can proceed without the latest parts if not.

- Our aim is, rather than just use an available prediction tool, to delve into the code to see how each step works. By analysing each part of the code, we wish to explain what important choices are made, and to which extent they influence the model's performance.

[1]: https://www.pyimagesearch.com/2019/12/09/how-to-install-tensorflow-2-0-on-ubuntu/
[2]: https://towardsdatascience.com/installing-tensorflow-with-cuda-cudnn-and-gpu-support-on-windows-10-60693e46e781

# Analysing the code
## Imports

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import sys
if '../../../embeddings' not in sys.path:
    sys.path.append('../../../embeddings')

from seq2tensor import s2t
import keras

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Embedding, LSTM, Bidirectional, BatchNormalization, merge, add
from keras.layers.core import Flatten, Reshape
from keras.layers.merge import Concatenate, concatenate, subtract, multiply
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D, AveragePooling1D, GlobalAveragePooling1D

from keras.optimizers import Adam,  RMSprop

import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

A certain number of libraries need to be imported for the network to be trained. In particular, tensorflow is an open-source platform dedicated for machine learning. Keras is a library enabling us to interact with tensorflow in a simple manner. Thanks to theses libraries, a variety of layers and transforms (such as convolutional layers, pooling layers, etc.) used in neural networks are directly available, without having to code them.

## GPU 

In [4]:
def get_session(gpu_fraction=0.25):
    '''Assume that you have 6GB of GPU memory and want to allocate ~2GB'''

    num_threads = os.environ.get('OMP_NUM_THREADS')
    #gpu_options = tf.compat.v1.GPUOptions(gpu_fraction)
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
#KTF.set_session(get_session())

import numpy as np
from tqdm import tqdm


#from keras.layers import Input, CuDNNGRU
from keras.layers import Input, GRU
#from tensorflow.compat.v1.keras.layers import CuDNNGRU
from numpy import linalg as LA
import scipy

Since most layers in neural networks rely on a high number of simple operations (essentially matrix multiplications), using a GPU rather than the CPU for computation is advised. GPUs are able to perform a high number of floating point operations, which is what is needed when training a network with a very high number of parameters.
Using a GPU rather than a CPU is not mandatory, but using a CPU will make the most resource-intensive task of training the model much longer.

## Access to input data and embedding

In [7]:
# Access to the protein sequences
cur_path = os.path.abspath('')
id2seq_file = os.path.relpath('..\\..\\..\\yeast\\preprocessed\\protein.dictionary.tsv', cur_path)

id2index = {}
seqs = []
index = 0
for line in open(id2seq_file):
    line = line.strip().split('\t')
    id2index[line[0]] = index
    seqs.append(line[1])
    index += 1
seq_array = []
id2_aid = {}
sid = 0

seq_size = 2000
emb_files = [os.path.relpath('..\\..\\..\\embeddings\\CTCoding_onehot.txt',cur_path), os.path.relpath('..\\..\\..\\embeddings\\string_vec5.txt',cur_path), os.path.relpath('..\\..\\..\\embeddings\\CTCoding_onehot.txt',cur_path), os.path.relpath('..\\..\\..\\embeddings\\vec5_CTC.txt',cur_path)]
use_emb = 0
hidden_dim = 25

- The next step is getting access to the input data. In our case, we only need the protein amino-acid sequences, which are located in the 'yeast' folder. Each line corresponds to the protein id, and its sequence. For example:  
P00044,MTEFKAGSAKKGATLFKTRCLQCHTVEKGGPHKVGPNLHGIFGRHSGQAEGYSYTDANIKKNVLWDENNMSEYLTNPKKYIPGTKMAFGGLKKEKDRNDLITYLKKACE
- If we want to train/test on another dataset, we will need to replace this file with our new dataset.
- This step has been modified from the original code, so as to function for the Windows OS. In particular, paths are written differently between unix/windows.

### Note
- The dataset used here is the Yeast dataset for binary PPI prediction provided in <cite>[Guo et al. 2008.][1]</cite>. It is a dataset widely used for benchmarking, and contains a balanced number of positive and negative samples (11 188 interaction cases in total).
- The quality and quantity of the dataset is essential in both training and testing the network.

[1]: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2396404/

## Epochs

In [8]:
n_epochs=2

- An epoch corresponds to one cycle through the full training dataset. If we stop the training after one epoch, the network will only see each protein once.
- Typically, the more epochs are used, the better the network will be trained. In this code, the number of epochs should be at least 50-100.

## Labels and results

In [9]:
# Labels for the dataset
ds_file = os.path.relpath('..\\..\\..\\yeast\\preprocessed\\protein.actions.tsv',cur_path)
#label_index = 2
label_index = -1
#Results file location
rst_file = os.path.relpath('results\\15k_onehot_cnn.txt',cur_path)
sid1_index = 0
sid2_index = 1

#Embedding and progress bars
seq2t = s2t(emb_files[use_emb])

max_data = -1
limit_data = max_data > 0
raw_data = []
skip_head = True
x = None
count = 0

for line in tqdm(open(ds_file)):
    if skip_head:
        skip_head = False
        continue
    line = line.rstrip('\n').rstrip('\r').split('\t')
    if id2index.get(line[sid1_index]) is None or id2index.get(line[sid2_index]) is None:
        continue
    if id2_aid.get(line[sid1_index]) is None:
        id2_aid[line[sid1_index]] = sid
        sid += 1
        seq_array.append(seqs[id2index[line[sid1_index]]])
    line[sid1_index] = id2_aid[line[sid1_index]]
    if id2_aid.get(line[sid2_index]) is None:
        id2_aid[line[sid2_index]] = sid
        sid += 1
        seq_array.append(seqs[id2index[line[sid2_index]]])
    line[sid2_index] = id2_aid[line[sid2_index]]
    raw_data.append(line)
    if limit_data:
        count += 1
        if count >= max_data:
            break
print (len(raw_data))


len_m_seq = np.array([len(line.split()) for line in seq_array])
avg_m_seq = int(np.average(len_m_seq)) + 1
max_m_seq = max(len_m_seq)
print (avg_m_seq, max_m_seq)

dim = seq2t.dim
seq_tensor = np.array([seq2t.embed_normalized(line, seq_size) for line in tqdm(seq_array)])

seq_index1 = np.array([line[sid1_index] for line in tqdm(raw_data)])
seq_index2 = np.array([line[sid2_index] for line in tqdm(raw_data)])

print(seq_index1[:10])

class_map = {'0':1,'1':0}
print(class_map)
class_labels = np.zeros((len(raw_data), 2))
for i in range(len(raw_data)):
    class_labels[i][class_map[raw_data[i][label_index]]] = 1.

11188it [00:00, 386813.34it/s]
 12%|█████████▌                                                                   | 312/2497 [00:00<00:00, 3097.38it/s]

11187
2 1


100%|███████████████████████████████████████████████████████████████████████| 11187/11187 [00:00<00:00, 3739077.13it/s]

[ 0  2  4  6  8 10 10 13 15 17]
{'0': 1, '1': 0}


- For both the training and test sets, labels are needed to either make the network, or analyse its performance.
- As for the protein sequences, the labels are in the 'yeast' folder. Each line contains the two protein ids, and wether they should interact or not. For example, 'Q08949 P38089 1' means that both proteins interact.
- The result of the testing phase will be written in the 'results/' folder. Again, for this example, the paths were modified from the original code to work for Windows.
- The rest of this code consists in checks for protein lenghts, number of proteins in the dataset, adding progression bars, etc.

## Building the model

In [10]:
def build_model():
    seq_input1 = Input(shape=(seq_size, dim), name='seq1')
    seq_input2 = Input(shape=(seq_size, dim), name='seq2')
    l1=Conv1D(hidden_dim, 3)
    #r1=Bidirectional(CuDNNGRU(hidden_dim, return_sequences=True))
    r1=Bidirectional(GRU(hidden_dim, return_sequences=True))
    l2=Conv1D(hidden_dim, 3)
    #r2=Bidirectional(CuDNNGRU(hidden_dim, return_sequences=True))
    r2=Bidirectional(GRU(hidden_dim, return_sequences=True))
    l3=Conv1D(hidden_dim, 3)
    #r3=Bidirectional(CuDNNGRU(hidden_dim, return_sequences=True))
    r3=Bidirectional(GRU(hidden_dim, return_sequences=True))
    l4=Conv1D(hidden_dim, 3)
    #r4=Bidirectional(CuDNNGRU(hidden_dim, return_sequences=True))
    r4=Bidirectional(GRU(hidden_dim, return_sequences=True))
    l5=Conv1D(hidden_dim, 3)
    #r5=Bidirectional(CuDNNGRU(hidden_dim, return_sequences=True))
    r5=Bidirectional(GRU(hidden_dim, return_sequences=True))
    l6=Conv1D(hidden_dim, 3)
    s1=MaxPooling1D(3)(l1(seq_input1))
    s1=concatenate([r1(s1), s1])
    s1=MaxPooling1D(3)(l2(s1))
    s1=concatenate([r2(s1), s1])
    s1=MaxPooling1D(3)(l3(s1))
    s1=concatenate([r3(s1), s1])
    s1=MaxPooling1D(3)(l4(s1))
    s1=concatenate([r4(s1), s1])
    s1=MaxPooling1D(3)(l5(s1))
    s1=concatenate([r5(s1), s1])
    s1=l6(s1)
    s1=GlobalAveragePooling1D()(s1)
    s2=MaxPooling1D(3)(l1(seq_input2))
    s2=concatenate([r1(s2), s2])
    s2=MaxPooling1D(3)(l2(s2))
    s2=concatenate([r2(s2), s2])
    s2=MaxPooling1D(3)(l3(s2))
    s2=concatenate([r3(s2), s2])
    s2=MaxPooling1D(3)(l4(s2))
    s2=concatenate([r4(s2), s2])
    s2=MaxPooling1D(3)(l5(s2))
    s2=concatenate([r5(s2), s2])
    s2=l6(s2)
    s2=GlobalAveragePooling1D()(s2)
    merge_text = multiply([s1, s2])
    x = Dense(100, activation='linear')(merge_text)
    x = keras.layers.LeakyReLU(alpha=0.3)(x)
    x = Dense(int((hidden_dim+7)/2), activation='linear')(x)
    x = keras.layers.LeakyReLU(alpha=0.3)(x)
    main_output = Dense(2, activation='softmax')(x)
    merge_model = Model(inputs=[seq_input1, seq_input2], outputs=[main_output])
    return merge_model

- This function is the core of the program. It corresponds to the structure of the neural network that is built (and shown below). In principle, since the idea is to predict protein interactions, it takes two protein sequences, and uses a Siamese Network in which each protein sequence goes through a separate number of layers. However, each subnetwork is identical, meaning that the parameters and weights are shared, and updated in the same way. The ouput of each subnetwork is then regrouped, then transformed into a real normalized value, indicating a prediction score for that interaction.
- In practice, each layer can be coded into a single line, as shown above. For example, Conv1D(n, m) performs a one-dimensional convolution with n output filters, and a kernel size of m.

#### Note
- As opposed to image prediction, protein-protein interaction prediction can be more difficult to visualize. In general we use similar architectures than those used in image prediction for example. The design of the architecture, the number of layers, epochs, etc. has an influence on the performance of the network. The exploration of the different hyper-parameters is not easy, but can be automated.

![title](images/PIPR_Architecture.png)

## Learning Options

In [11]:
batch_size1 = 256
adam = Adam(lr=0.001, amsgrad=True, epsilon=1e-6)
rms = RMSprop(lr=0.001)

- A certain number of choices are needed for training. 
- In principle, the training step involves the model making predictions on the training data, and using the error of these predictions to update the model, so that the error is reduced.
- The batch size corresponds to the number of samples that are passed through the network together, before updating the internal model parameters. The batch size can take values from 1 (updates are computed after each interaction example) to the number of elements in the training dataset (errors are computed for each example, but the model is updated only after the whole dataset has been passed through the network). Here we have a compromise between those two cases, aiming at balance between efficiency and robustness.
- The learning rate corresponds to the step used for the update of model, when minimizing the loss function.

## Cross-validation

In [30]:
from sklearn.model_selection import KFold, ShuffleSplit
#kf = KFold(n_splits=5, shuffle=True)
kf = KFold(n_splits=2, shuffle=True)
tries = 5
cur = 0
recalls = []
accuracy = []
total = []
total_truth = []
train_test = []
for train, test in kf.split(class_labels):
    if np.sum(class_labels[train], 0)[0] > 0.8 * len(train) or np.sum(class_labels[train], 0)[0] < 0.2 * len(train):
        continue
    train_test.append((train, test))
    cur += 1
    if cur >= tries:
        break

The last parameters that are needed for training is the split of the original dataset into training and testing.
- In our example, we perform a 5-fold cross-validation. The dataset is divided into 5 block, each block is used once for testing while the others are used for training. This means that 5 separate training phases will be performed, with 5 different training sets. We can look at the training/test set size below: 

In [21]:
print ("Number of training phases:",len(train_test))
dataset1 = train_test[0]
print("Number of samples in first training set: ",len(dataset1[0]))
print("Number of samples in first test set: ",len(dataset1[1]))

Number of training phases: 5
Number of samples in first training set:  8949
Number of samples in first test set:  2238


In [28]:
print("Example of protein sequence hot-encoding:\n",seq_tensor[seq_index1[dataset1[1][0]]])

Example of protein sequence hot-encoding:
 [[0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Training and testing

In [31]:
#copy below
num_hit = 0.
num_total = 0.
num_pos = 0.
num_true_pos = 0.
num_false_pos = 0.
num_true_neg = 0.
num_false_neg = 0.

for train, test in train_test:
    merge_model = None
    merge_model = build_model()
    adam = Adam(lr=0.001, amsgrad=True, epsilon=1e-6)
    rms = RMSprop(lr=0.001)
    merge_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    merge_model.fit([seq_tensor[seq_index1[train]], seq_tensor[seq_index2[train]]], class_labels[train], batch_size=batch_size1, epochs=n_epochs)
    #result1 = merge_model.evaluate([seq_tensor1[test], seq_tensor2[test]], class_labels[test])
    pred = merge_model.predict([seq_tensor[seq_index1[test]], seq_tensor[seq_index2[test]]])
    for i in range(len(class_labels[test])):
        num_total += 1
        if np.argmax(class_labels[test][i]) == np.argmax(pred[i]):
            num_hit += 1
        if class_labels[test][i][0] > 0.:
            num_pos += 1.
            if pred[i][0] > pred[i][1]:
                num_true_pos += 1
            else:
                num_false_neg += 1
        else:
            if pred[i][0] > pred[i][1]:
                num_false_pos += 1
            else:
                num_true_neg += 1
    '''accuracy = num_hit / num_total
    prec = num_true_pos / (num_true_pos + num_false_pos)
    recall = num_true_pos / num_pos
    spec = num_true_neg / (num_true_neg + num_false_neg)
    f1 = 2. * prec * recall / (prec + recall)
    mcc = (num_true_pos * num_true_neg - num_false_pos * num_false_neg) / ((num_true_pos + num_true_neg) * (num_true_pos + num_false_neg) * (num_false_pos + num_true_neg) * (num_false_pos + num_false_neg)) ** 0.5
    print (accuracy, prec, recall, spec, f1, mcc)
    '''
print("finished")

Epoch 1/2
5593/5593 [==============================] - 100s 18ms/step - loss: 0.6933 - accuracy: 0.4960
Epoch 2/2
5593/5593 [==============================] - 86s 15ms/step - loss: 0.6930 - accuracy: 0.5171
Epoch 1/2
5594/5594 [==============================] - 92s 16ms/step - loss: 0.6936 - accuracy: 0.4993
Epoch 2/2
5594/5594 [==============================] - 84s 15ms/step - loss: 0.6932 - accuracy: 0.4971
finished


- The final step is doing the actual training/testing. Simple performance indicators are computed, such as the specificity, accuracy, recall f1-score and mcc.
- The accuracy is shown after each epoch, it should increase progressively. After a certain number of epochs, the performance should stabilize. After 100 epochs, for this network, the performance is quite high (f1-score around 97%).
- Note : for reasonable computing time, GPU is required, as well as a fair amount of memory (depending on the dataset size).

In [34]:
    
accuracy = num_hit / num_total
prec = num_true_pos / (num_true_pos + num_false_pos)
recall = num_true_pos / num_pos
spec = num_true_neg / (num_true_neg + num_false_neg)
f1 = 2. * prec * recall / (prec + recall)
mcc = (num_true_pos * num_true_neg - num_false_pos * num_false_neg) / ((num_true_pos + num_true_neg) * (num_true_pos + num_false_neg) * (num_false_pos + num_true_neg) * (num_false_pos + num_false_neg)) ** 0.5
print("Accuracy:",accuracy)
print("Precision:", prec)
print("Recall:",recall)
print("F1-score:", f1)

with open(rst_file, 'w') as fp:
    fp.write('acc=' + str(accuracy) + '\tprec=' + str(prec) + '\trecall=' + str(recall) + '\tspec=' + str(spec) + '\tf1=' + str(f1) + '\tmcc=' + str(mcc))

Accuracy: 0.5180119781889694
Precision: 0.5230981383589979
Recall: 0.40693724298229933
F1-score: 0.45776347546259055


## Conclusion
We have successfully trained and tested a Deep Network for Protein-Protein Interaction Prediction with PIPR. The network we are currently working on has a similar architecture, namely a Siamese Network. However, we introduce a certain number of preprocessing steps for the training dataset, as well as another approach in the way the convolution layers are applied, then merged (an illustration of our architecture is shown below). Our model is in its last phase of testing, and will soon be available.
![title](images/Imprint_Architecture.png)